# Phase Matching Explanation
# Explicação de Casamento de Fase

## Summary / Resumo

This notebook demonstrates phase matching optimization to find the optimal metasurface layout for target TE/TM phase profiles.

Este notebook demonstra otimização de casamento de fase para encontrar o layout ótimo de metassuperfície para perfis de fase TE/TM alvo.

## Inputs / Entradas

**Required files / Arquivos necessários:**
1. Cleaned library with phase_TE, phase_TM, L_x, L_y columns
   Biblioteca limpa com colunas phase_TE, phase_TM, L_x, L_y
2. Target TE phase map (2D array)
   Mapa de fase TE alvo (array 2D)
3. Target TM phase map (2D array)
   Mapa de fase TM alvo (array 2D)

**Optional / Opcional:**
- Height column (H) for height-based filtering
  Coluna de altura (H) para filtragem baseada em altura

In [ ]:
# Import required packages / Importar pacotes necessários
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Add src to path / Adicionar src ao caminho
repo_root = Path.cwd().parent.parent
sys.path.insert(0, str(repo_root / 'src'))

from meta_library import phase_matching

print('✓ Packages loaded successfully / Pacotes carregados com sucesso')

## Configuration / Configuração

In [ ]:
# Configuration / Configuração
library_file = repo_root / 'library_processed' / 'example_library.csv'  # Update / Atualize
output_dir = repo_root / 'results' / 'meta_library' / 'phase_matching' / 'notebook_example'
output_dir.mkdir(parents=True, exist_ok=True)

# For demonstration, we'll create synthetic target phases
# Para demonstração, criaremos fases alvo sintéticas
target_shape = (50, 50)  # pixels

# Phase matching options / Opções de casamento de fase
use_height_filter = False
target_height = None  # Set to specific value if use_height_filter=True

print(f'Library file: {library_file}')
print(f'Target shape: {target_shape}')
print(f'Output directory: {output_dir}')

## Run / Execução

### Step 1: Load Library / Passo 1: Carregar Biblioteca

In [ ]:
# Load library / Carregar biblioteca
if library_file.suffix == '.parquet':
    df = pd.read_parquet(library_file)
else:
    df = pd.read_csv(library_file)

print(f'Loaded {len(df)} rows')
print(f'\nRequired columns present:')
for col in ['phase_TE', 'phase_TM', 'L_x', 'L_y']:
    print(f'  {col}: {col in df.columns}')

# Show phase coverage / Mostrar cobertura de fase
if 'phase_TE' in df.columns and 'phase_TM' in df.columns:
    print(f'\nPhase coverage / Cobertura de fase:')
    print(f'  TE: [{df.phase_TE.min():.3f}, {df.phase_TE.max():.3f}] rad')
    print(f'  TM: [{df.phase_TM.min():.3f}, {df.phase_TM.max():.3f}] rad')

### Step 2: Create/Load Target Phases / Passo 2: Criar/Carregar Fases Alvo

For this demonstration, we'll create synthetic target phases. In practice, these would come from hologram design or other optimization.

Para esta demonstração, criaremos fases alvo sintéticas. Na prática, elas viriam de design de holograma ou outra otimização.

In [ ]:
# Create synthetic target phases / Criar fases alvo sintéticas
# Example: quadratic phase pattern / Exemplo: padrão de fase quadrático
x = np.linspace(-1, 1, target_shape[1])
y = np.linspace(-1, 1, target_shape[0])
X, Y = np.meshgrid(x, y)

# Quadratic phase for TE / Fase quadrática para TE
target_phase_te = np.pi * (X**2 + Y**2)

# Different quadratic phase for TM / Fase quadrática diferente para TM
target_phase_tm = 0.8 * np.pi * (X**2 + Y**2) + 0.5

# Normalize to library range if needed / Normalizar para o range da biblioteca se necessário
if 'phase_TE' in df.columns:
    te_min, te_max = df.phase_TE.min(), df.phase_TE.max()
    tm_min, tm_max = df.phase_TM.min(), df.phase_TM.max()
    
    target_phase_te = te_min + (target_phase_te - target_phase_te.min()) / \
                      (target_phase_te.max() - target_phase_te.min()) * (te_max - te_min)
    target_phase_tm = tm_min + (target_phase_tm - target_phase_tm.min()) / \
                      (target_phase_tm.max() - target_phase_tm.min()) * (tm_max - tm_min)

# Save targets / Salvar alvos
np.save(output_dir / 'target_phase_te.npy', target_phase_te)
np.save(output_dir / 'target_phase_tm.npy', target_phase_tm)

# Visualize targets / Visualizar alvos
fig, axes = plt.subplots(1, 2, figsize=(10, 4))

im0 = axes[0].imshow(target_phase_te, cmap='twilight')
axes[0].set_title('Target Phase TE')
plt.colorbar(im0, ax=axes[0])

im1 = axes[1].imshow(target_phase_tm, cmap='twilight')
axes[1].set_title('Target Phase TM')
plt.colorbar(im1, ax=axes[1])

plt.tight_layout()
plt.savefig(output_dir / 'target_phases.png', dpi=300, bbox_inches='tight')
plt.show()

print('✓ Target phases created / Fases alvo criadas')

### Step 3: Perform Phase Matching / Passo 3: Realizar Casamento de Fase

The algorithm finds the library entry that minimizes quadratic error for each pixel:

O algoritmo encontra a entrada da biblioteca que minimiza o erro quadrático para cada pixel:

$$\text{error}_{i,j} = \sqrt{(\phi_{TE}^{target} - \phi_{TE}^{lib})^2 + (\phi_{TM}^{target} - \phi_{TM}^{lib})^2}$$

In [ ]:
# Perform phase matching / Realizar casamento de fase
layout_lx, layout_ly, error_map = phase_matching.perform_phase_matching(
    df,
    target_phase_tm=target_phase_tm,
    target_phase_te=target_phase_te,
    use_height=use_height_filter,
    target_height=target_height
)

print('✓ Phase matching complete / Casamento de fase completo')
print(f'\nError statistics / Estatísticas de erro:')
print(f'  Mean RMS: {error_map.mean():.6f} rad')
print(f'  Max RMS:  {error_map.max():.6f} rad')
print(f'  Min RMS:  {error_map.min():.6f} rad')

### Step 4: Save Results / Passo 4: Salvar Resultados

In [ ]:
# Save layout outputs / Salvar saídas de layout
saved_paths = phase_matching.save_layout_outputs(
    layout_lx, layout_ly, error_map,
    out_dir=output_dir,
    prefix='layout'
)

print('✓ Results saved / Resultados salvos')
print(f'\nSaved files / Arquivos salvos:')
for key, path in saved_paths.items():
    print(f'  {key}: {path.name}')

## Results / Resultados

### Visualization / Visualização

In [ ]:
# Comprehensive visualization / Visualização abrangente
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Row 1: Targets
im0 = axes[0, 0].imshow(target_phase_te, cmap='twilight')
axes[0, 0].set_title('Target Phase TE')
plt.colorbar(im0, ax=axes[0, 0])

im1 = axes[0, 1].imshow(target_phase_tm, cmap='twilight')
axes[0, 1].set_title('Target Phase TM')
plt.colorbar(im1, ax=axes[0, 1])

axes[0, 2].axis('off')
axes[0, 2].text(0.5, 0.5, f'Mean Error:\n{error_map.mean():.4f} rad',
               ha='center', va='center', fontsize=14, weight='bold',
               bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# Row 2: Results
im3 = axes[1, 0].imshow(layout_lx, cmap='viridis')
axes[1, 0].set_title('Layout L_x (nm)')
plt.colorbar(im3, ax=axes[1, 0])

im4 = axes[1, 1].imshow(layout_ly, cmap='viridis')
axes[1, 1].set_title('Layout L_y (nm)')
plt.colorbar(im4, ax=axes[1, 1])

im5 = axes[1, 2].imshow(error_map, cmap='hot')
axes[1, 2].set_title('RMS Error Map')
plt.colorbar(im5, ax=axes[1, 2])

plt.tight_layout()
plt.savefig(output_dir / 'phase_matching_complete.png', dpi=300, bbox_inches='tight')
plt.show()

print('✓ Visualization complete / Visualização completa')

### Interpretation / Interpretação

**Layout Maps (L_x, L_y) / Mapas de Layout:**
- Show the optimal nanopilar dimensions for each pixel
- Mostram as dimensões ótimas do nanopilar para cada pixel
- Can be used directly for fabrication
- Podem ser usados diretamente para fabricação

**Error Map / Mapa de Erro:**
- Shows phase matching quality at each pixel
- Mostra qualidade de casamento de fase em cada pixel
- Lower values indicate better matching
- Valores menores indicam melhor casamento
- Hot spots may indicate limitations in library coverage
- Pontos quentes podem indicar limitações na cobertura da biblioteca

## Connection to Holography Pipeline / Conexão com Pipeline de Holografia

The phase matching module integrates with the holography pipeline:

O módulo de casamento de fase integra com o pipeline de holografia:

1. **Hologram Design** (from `src/holography/gs_asm.py`)
   - Generates target phase maps for desired images
   - Gera mapas de fase alvo para imagens desejadas

2. **Phase Matching** (this module)
   - Finds metasurface layout to realize target phases
   - Encontra layout de metassuperfície para realizar fases alvo

3. **Fabrication**
   - Layout maps (L_x, L_y) define nanopilar geometries
   - Mapas de layout (L_x, L_y) definem geometrias de nanopilar

### Example Integration / Exemplo de Integração

```python
# 1. Design hologram
from holography import gs_asm
target_image = load_image('logo.png')
phase_te = gs_asm.gerchberg_saxton_angular_spectrum(target_image, ...)
phase_tm = phase_te  # or different for polarization multiplexing

# 2. Perform phase matching
layout_lx, layout_ly, error = phase_matching.perform_phase_matching(
    library_df, phase_tm, phase_te
)

# 3. Export for fabrication
np.savetxt('layout_lx.csv', layout_lx)
np.savetxt('layout_ly.csv', layout_ly)
```

## Reproducibility / Reprodutibilidade

### Using CLI / Usando CLI

```bash
python src/cli/run_phase_matching.py \
  --library library_cleaned.csv \
  --target-te target_phase_te.npy \
  --target-tm target_phase_tm.npy \
  --preview \
  --experiment notebook_example
```

### Python API

```python
from meta_library import phase_matching
import pandas as pd
import numpy as np

df = pd.read_csv('library_cleaned.csv')
target_te = np.load('target_phase_te.npy')
target_tm = np.load('target_phase_tm.npy')

lx, ly, err = phase_matching.perform_phase_matching(
    df, target_tm, target_te
)
phase_matching.save_layout_outputs(lx, ly, err, 'output')
```